In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as logreg
from sklearn import metrics

In [4]:
import psycopg2
con = psycopg2.connect(database="mid_term_project", user="lhl_student", password="lhl_student", host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")
cur = con.cursor()

In [26]:
flight2 = pd.read_sql_query('''SELECT * FROM flights where fl_date like '%-01-%' LIMIT 1000;''', con)

In [31]:
for value in flight2.loc[:,'dep_time'].head():
    if value in range(900, 1200):
        print(value)

1013.0


In [114]:
flight_full = pd.read_sql_query('''SELECT * FROM flights where fl_date like '%-12-%' LIMIT 100000;''', con)

In [ ]:
flightvariables =  ['fl_date', 'op_unique_carrier','tail_num', 'origin_airport_id', 'dest_airport_id', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance', 'arr_delay']

In [24]:
flight_na = flight.dropna()

In [25]:
flight_na.head()

,fl_date,op_unique_carrier,tail_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay
10,2018-12-01,NK,N602NK,14100,12889,1945,2244,359.0,2176.0,-29.0
11,2018-12-01,NK,N602NK,12889,14100,1110,1855,285.0,2176.0,1.0
12,2018-12-01,NK,N507NK,10397,11697,1739,1930,111.0,581.0,-15.0
13,2018-12-01,NK,N507NK,12892,10397,930,1649,259.0,1947.0,-35.0
14,2018-12-01,NK,N512NK,10397,12892,1120,1330,310.0,1947.0,11.0


In [32]:
flight_na['is_arr_delayed'] = 0

<ipython-input-32-55bb6c02d91a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_na['is_arr_delayed'] = 0


In [34]:
flight_na['is_arr_delayed'].loc[flight_na['arr_delay'] > 0] = 1

G:\Programming\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-34-a3d368983c6c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_na['is_arr_delayed'].loc[flight_na['arr_delay'] > 0] = 1


In [43]:
# Bad Strategy. Use getdummy
#carrier_list = flight_na['op_unique_carrier'].unique().tolist()

In [47]:
#carrier_code = pd.Series(carrier_list).astype('category').cat.codes.values.tolist()

In [45]:
#carrier_list.todict()

AttributeError: 'list' object has no attribute 'todict'

In [53]:
flight_na = flight_na.drop('arr_delay', axis = 1)

In [78]:
flight_dum = pd.get_dummies(flight_na, columns = ['op_unique_carrier'])

In [79]:
flight_dt = pd.to_datetime(flight_dum['fl_date'])

In [81]:
flight_dum['year'] =  pd.to_datetime(flight_dum['fl_date']).dt.year
flight_dum['month'] =  pd.to_datetime(flight_dum['fl_date']).dt.month
flight_dum['day'] =  pd.to_datetime(flight_dum['fl_date']).dt.day
flight_dum['weekday'] =  pd.to_datetime(flight_dum['fl_date']).dt.dayofweek

In [85]:
flight_dum = flight_dum.drop('fl_date', axis = 1)

In [94]:
flight_dum.shape

(99092, 37)

In [ ]:
Datetime64[ns]

For extracting year,month and day to new column,follow the code:

df['year'] = df['date'].dt.year

df['month'] = df['date'].dt.month

df['day'] = df['date'].dt.day

In [68]:
pd.to_datetime(flight_dum['fl_date']).shape

(5,)

In [91]:
flight_dum.to_csv('baselineDF.csv')

# Log Reg 

In [92]:
X = flight_dum.drop('is_arr_delayed', axis = 1)
y = flight_dum['is_arr_delayed']


In [112]:
from sklearn.metrics import roc_auc_score

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [98]:
y_train.shape

(79273,)

In [100]:
from sklearn.model_selection import GridSearchCV

In [103]:
clf = logreg(max_iter = 10000).fit(X_train,y_train)

In [106]:
Y_predict = clf.predict(X_test)

In [109]:
from sklearn.metrics import accuracy_score

In [110]:
accuracy_score(y_test, Y_predict)

0.659114990665523

In [113]:
roc_auc_score(y_test, Y_predict)

0.5012803806502517

In [120]:
flight_fullna = flight_full['arr_delay'].dropna()

In [121]:
flight_fullna.shape

(99092,)

In [117]:
flight_dum.shape

(99092, 37)

In [140]:
flight_fullna.to_csv('baselineDF_full.csv')

In [141]:
flight_fullna = flight_full.dropna(axis=0, subset = ['arr_delay'])
flight_fullna.shape

(99092, 42)